<a href="https://colab.research.google.com/github/Shahin2512/Personalized-Mentor-Recommendation-System/blob/main/Personalized_Mentor_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas scikit-learn


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer


In [3]:
mentors = pd.read_csv('mentors_data.csv')
aspirants = pd.read_csv('aspirants_data.csv')

import ast
mentors['preferred_subjects'] = mentors['preferred_subjects'].apply(ast.literal_eval)
mentors['target_colleges'] = mentors['target_colleges'].apply(ast.literal_eval)
aspirants['preferred_subjects'] = aspirants['preferred_subjects'].apply(ast.literal_eval)
aspirants['target_colleges'] = aspirants['target_colleges'].apply(ast.literal_eval)


In [4]:
mlb_subjects = MultiLabelBinarizer()
mlb_colleges = MultiLabelBinarizer()
learning_styles_dummies = pd.get_dummies(mentors['learning_style'])

subject_encoded = mlb_subjects.fit_transform(mentors['preferred_subjects'])
college_encoded = mlb_colleges.fit_transform(mentors['target_colleges'])

mentor_features = np.hstack([subject_encoded, college_encoded, learning_styles_dummies])


In [5]:
aspirant = aspirants.iloc[0]

asp_subjects = mlb_subjects.transform([aspirant['preferred_subjects']])
asp_colleges = mlb_colleges.transform([aspirant['target_colleges']])
asp_learning = pd.get_dummies(pd.Series([aspirant['learning_style']]))
asp_learning = asp_learning.reindex(columns=learning_styles_dummies.columns, fill_value=0)

asp_features = np.hstack([asp_subjects, asp_colleges, asp_learning.values])


In [6]:
similarity_scores = cosine_similarity(asp_features, mentor_features).flatten()
top_n = 3
top_indices = similarity_scores.argsort()[::-1][:top_n]

recommended_mentors = mentors.iloc[top_indices][['mentor_id', 'name', 'preferred_subjects', 'target_colleges', 'learning_style']]
print("Recommended Mentors:")
print(recommended_mentors)


Recommended Mentors:
    mentor_id      name        preferred_subjects  \
16         17  Mentor Q    [Logical, Legal, Math]   
15         16  Mentor P  [Math, Logical, English]   
0           1  Mentor A    [Legal, Math, English]   

                   target_colleges learning_style  
16               [NLSIU Bangalore]       Auditory  
15  [NLU Jodhpur, NLSIU Bangalore]       Auditory  
0   [NLSIU Bangalore, NLU Kolkata]       Auditory  


In [7]:
# Simulated feedback (1 = liked, 0 = disliked)
feedback = pd.Series([1, 0, 1])


feedback_df = pd.DataFrame({
    'mentor': recommended_mentors['name'].values,
    'similarity': similarity_scores[top_indices],
    'feedback': feedback
})

print("\nFeedback Analysis:")
print(feedback_df)

print("\nAverage similarity for liked mentors:", feedback_df[feedback_df['feedback'] == 1]['similarity'].mean())
print("Average similarity for disliked mentors:", feedback_df[feedback_df['feedback'] == 0]['similarity'].mean())



Feedback Analysis:
     mentor  similarity  feedback
0  Mentor Q    0.800000         1
1  Mentor P    0.730297         0
2  Mentor A    0.730297         1

Average similarity for liked mentors: 0.7651483716701107
Average similarity for disliked mentors: 0.7302967433402215
